In [ ]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                                                         #
#                          04/2021     Adrien Poindron    #
###########################################################

In [49]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

# Imports

In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib.patches import Ellipse
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit

In [4]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import matplotlib.patches as patches

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [5]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [22]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../Functions/210426_stopping_power.py"
%run "../Functions/turbo_colormap.py"

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e4*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [8]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : H:/Universite_Simus/20210608/EGMol07/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [2, 19, 28, 36]
> slashcond | -2
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 70


In [9]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['EGMol07', 'EGMol08', 'EGMol09', 'EGMol10', 'EGMol11', 'EGMol12', 'EGMol13']
> condition names ['EGMol']
> number of points 7
> N_ions = 1024
> e_GMol = 50


In [10]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg,mode='stopping_power') 

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 7
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05', 'Try06', 'Try07', 'Try08', 'Try09']
Hello
Point n° 0
00 - 00  >  H:/Universite_Simus/20210608\EGMol07\Try00/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
00 - 01  >  H:/Universite_Simus/20210608\EGMol07\Try01/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
00 - 02  >  H:/Universite_Simus/20210608\EGMol07\Try02/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat', 'Temp_SimuTyp

02 - 05  >  H:/Universite_Simus/20210608\EGMol09\Try05/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
02 - 06  >  H:/Universite_Simus/20210608\EGMol09\Try06/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
02 - 07  >  H:/Universite_Simus/20210608\EGMol09\Try07/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
02 - 08  >  H:/Universite_Simus/20210

05 - 01  >  H:/Universite_Simus/20210608\EGMol12\Try01/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
05 - 02  >  H:/Universite_Simus/20210608\EGMol12\Try02/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
05 - 03  >  H:/Universite_Simus/20210608\EGMol12\Try03/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
05 - 04  >  H:/Universite_Simus/20210

In [11]:
# Where SNR for each try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))

print(len(trig_detection))

# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

[0. 0. 0. 0. 0. 0. 0.]
7
1.0


In [12]:
# Voltages associated with each condition
# In names and files each voltage is associated with numbers
# For instance on epoint is labeled DC03_RF18
# You need to know which voltage is DC03 and RF18

# You have to look on the Fortran code, example :
#    !dec$ elseif defined(Vrf18)
#    double precision  , parameter :: V_rf  = 7.179d1
#    character(len=100), parameter :: str_extra3 = 'q=0.666'
# Here RF18 is 71.79 V, q = 0.666

Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]
len(Udc),len(Urf)

(20, 21)

In [15]:
(len(points_and_coord),13,total_len,3)

(7, 13, 19817, 3)

In [16]:
shape(ti)

(6, 3)

In [14]:
# recovery temperature
# calculate T final SimuType0
# Determination error on temperature
# with measurement of the variation in the simulations

my_order = [0,2,1]

# retrieve time array
point_numb = 3
point_try = 0
plot_subdir = all_subdir[point_numb*13+point_try%13] # point address
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]

total_len = 0
cut_pts = [0]
simu_time = []
for j,k in enumerate(my_order):
    # tt, T_CM, T_aux, PM
    ta, te, ti, to = load_T_and_PM_simu(plot_subdir+'/'+'Temp'+sort(onlyfiles)[k].strip('.dat')[4:])
    total_len += len(ta)
    # print(len(ta))
    cut_pts.append(total_len)
    simu_time.extend(ta)
# print(temp_pts)

# retrieve temperatures
T_aux_all = np.zeros((len(points_and_coord),13,total_len,3))
for point_numb,point_name in enumerate(points_and_coord): # DCxx_RFyy [xx,yy]
    for point_try in range(13):
        plot_subdir = all_subdir[point_numb*13+point_try%13] # point address
        onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
        for j,k in enumerate(my_order):
            ta, te, ti, to = load_T_and_PM_simu(plot_subdir+'/'+'Temp'+sort(onlyfiles)[k].strip('.dat')[4:])
            T_aux_all[point_numb,point_try,cut_pts[j]:cut_pts[j+1],:] = ti
    print(point_numb)
        # tt, T_CM, T_aux, PM

ValueError: could not broadcast input array from shape (6,3) into shape (17,3)

# Plots
Nuage d'ions (XYZ) et température

## Visualise one specific point, and one case

# One try study

In [23]:
# plot ion cloud XYZ
plot_point = 0
plot_try = 0
print(plot_subdir)
# pick the right xyz files
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')
auxr_LC,auxv_LC,auxa_LC, auxv_LC_avg = load_xyz_init_bin_DP(file_name)
len(auxr_LC[0])

H:/Universite_Simus/20210608\EGMol07\Try00
1024
b'1000'
1024
b'1000'


D:\CIMLuser\Documents\GitHub\GiantMol\Functions\210426_stopping_power.py:94: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  aux_info = genfromtxt(str_load[:xva_pos]+'Langevin_cooling'+'.info',comments='%',dtype=None)


1024

In [24]:
N_ions = 1024
n_dt = 500

In [29]:
# rep_name = 'K:/Universite_Simus/20210426/EGMol08/Try00/'

rep_name = 'H:/Universite_Simus/20210608/EGMol08/Try00/'
# rep_name = '/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/'
cond_name = 'Vrf0064_Udc0.2000D+01V'
# cond_name = 'Vrf0010_Udc0.5000D+02V'
print(rep_name)

# file = rep_name + 'Temp_SimuType0_N00128_Vrf0010_Udc0.5000D+00V_nt100'
file_SimuType0 = rep_name + f'Temp_SimuTypeQ_N{N_ions:05d}_' + cond_name + '_nt100'
print(file_SimuType0)
# time, T_CM, T_aux, samp_dE_dt, S_heating = load_T_and_dE_S_simu(file_SimuType0)
# print(samp_dE_dt)
# file_energies = rep_name + 'Energies_ndt100_vdt2'
# j_save_E, size_Energy_array, dE_dt = load_energies(file_energies,flag_print=1)
# full_dE_dt = reshape(dE_dt,(int(size_Energy_array),6),order='F')[:,3]
# print(full_dE_dt)

H:/Universite_Simus/20210608/EGMol08/Try00/
H:/Universite_Simus/20210608/EGMol08/Try00/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_nt100


In [30]:
str_load_Lan = rep_name + f'xva_3D_Harmo_N{N_ions:04d}_T500uK_F0.10D-19Kg_s_5'
# print(str_load_Lan)
# xva_pos = str_load_Lan.find('xva')
# print(str_load_Lan[:xva_pos]+'Langevin_cooling'+'.info')
# print(str_load_Lan+'.bin')
# plot_XYZ_Lan(str_load_Lan)

In [31]:
str_load_Ver = rep_name + f'xva_SimuTypeQ_N{N_ions:05d}_' + cond_name + '_D1.0_S1.0RFG'
print(str_load_Ver)
xva_pos = str_load_Ver.find('xva')
print(str_load_Ver[:xva_pos]+'Langevin_cooling'+'.info')
print(str_load_Ver+'.bin')
plot_XYZ(str_load_Ver)

H:/Universite_Simus/20210608/EGMol08/Try00/xva_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
H:/Universite_Simus/20210608/EGMol08/Try00/Langevin_cooling.info
H:/Universite_Simus/20210608/EGMol08/Try00/xva_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.bin
1024
b'1000'


D:\CIMLuser\Documents\GitHub\GiantMol\Functions\210426_stopping_power.py:94: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  aux_info = genfromtxt(str_load[:xva_pos]+'Langevin_cooling'+'.info',comments='%',dtype=None)


In [33]:
## Data loading with hardcode names
eta = 1.5e-19

plot_point = 0
plot_try = 0
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.1D-19'
dirname_hc = str(data0[plot_point][plot_try])+'/'
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

H:/Universite_Simus/20210608\EGMol07\Try00/
Temp_3D_Harmo_N1024_T500uK_F0.1D-19Kg_s_5


In [34]:
# Get T and fluo PM  All SimuType
onlyfiles = [f for f in listdir(rep_name) if isfile(join(rep_name, f)) and "SimuType" in f and ".dat" in f]
print(sort(onlyfiles))
only_init = [f for f in onlyfiles if "SimuTypeQ" in f]

['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV.dat'
 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV.dat']


In [39]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../Functions/210426_stopping_power.py"

In [35]:
renorm = 0.5*m_GM/C_e

In [43]:
onlyfiles[0].strip('.dat')

'Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV'

In [44]:
# Temperature and dE/dt

temps, temperature, fluo = plot_T_and_PM_Init_Inje_Evol(rep_name,onlyfiles[0].strip('.dat'),flag_plot=1,
                                    fig_name='popopo',
                                    xlim1=(-2,12),ylim1=(1e-5,8e4)) # ,ylim2=(-5e-12,5e-12)
# print(min(dE))
# savefig(rep_name+'T_dE')
# tt[1980],tt1[0]

010 eV
Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV
len save_T [55440]
[5.05000000e-07 6.11450868e-06 5.18197046e-03 1.13339581e-05
 4.71914098e+01 1.50491589e+00 7.17324773e+01]


In [45]:
# Delta Ec GMol en meV
# Gmol_data[0] est en (m/s)**2
# renorm inroduit les facteurs 0.5*M_Gmol et la conversion en eV
# 1000 converti en meV
print(Gmol_data[0]*renorm*1e3)
print(shape(Gmol_data[0]))

[[ -1.80463066  -2.77639033  -1.94052752  -3.21575813  -2.42044188
   -2.79731763  -1.79074365  -2.4657554   -3.71646736  -2.8099531 ]
 [ -2.65107843  -1.70209998  -3.73574812  -2.51513977  -8.89182848
   -7.87225996  -4.07798617  -3.43007062  -3.23672462  -2.57057709]
 [ -3.01878787 -15.53825811  -2.81636721  -7.79024349  -3.58884096
   -3.88916716 -27.62370279  -4.59685991  -6.18963436  -5.60792092]
 [ -6.89175657 -42.98435928  -5.88777095  -5.21925813  -5.22832477
   -4.92716899  -3.22491977 -11.23639114  -4.05999707  -4.98767456]
 [ -5.18468869  -6.7690575   -7.80988771 -14.24639933  -7.32398552
  -12.45167334  -4.30950144  -4.43647434 -13.15561932  -7.75440049]
 [-31.92672687 -17.59578163 -14.33560597 -24.50129147 -37.55751357
  -16.18784855 -17.75751388  -6.92779312  -7.43248472  -8.43089851]
 [-10.25791919 -34.04061335  -6.28858275 -11.03346905 -15.48092847
   -9.12194052  -6.49303142 -45.21284276  -9.87809394 -13.93395264]]
(7, 10)


In [46]:
my_EGmol_i = [0,0,0,0,0,0,0,50,10,8,6,4,2,1]
cm = pylab.get_cmap('turbo')

In [51]:
fname='deltaEc_10**4_amu'
fig, ax = plt.subplots(1, 1,num=fname,clear=True)
deltaEc_avg = []
for EGmol_i,deltaEc_EGmol_i in enumerate(Gmol_data[0]):
#     print(all_subdir[10*Egmol_i])
#     print(EGmol_i+7,deltaEc_EGmol_i)
    deltaEc_avg.append(mean(deltaEc_EGmol_i)*renorm*1e3)
#     print(deltaEc_avg)
    
    for v,w in enumerate(deltaEc_EGmol_i):
        ax.scatter(my_EGmol_i[EGmol_i+7],w*renorm*1e3,color=cm(EGmol_i/len(Gmol_data[0])))
ax.plot(my_EGmol_i[7:],deltaEc_avg)

ax.set_xlabel('$E_{GMol}^i$  [eV]')
ax.set_ylabel('$\Delta E_{GMol}$ [meV]')


# Change major ticks to show every 20.
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_major_locator(MultipleLocator(10))
# Change minor ticks to show every 5. (20/4 = 5)
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(4))
# Turn grid on for both major and minor ticks and style minor slightly
# differently.
ax.grid(which='major', color='#CCCCCC', linestyle='--')
ax.grid(which='minor', color='#CCCCCC', linestyle=':')

fig.set_size_inches(11.69, 8.27)
tight_layout()